In [20]:
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.losses import mse
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Input, Dense, Lambda, Conv2D, Flatten, Reshape, Conv2DTranspose
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [21]:
# Define the base directory where the dataset is located
base_dir = 'C:\\Users\\20203894\\Documents\\8p361'

# Define the model name
model_name = 'CVEA_Project_AI_version_'+'1'

# Define the file path for saving the model architecture
model_filepath = model_name + '.json'

# Define the file path for saving the model weights
weights_filepath = model_name + '_weights.hdf5'

# Batch size for training
batch_size = 32

# Image size
img_size = 96

# The number of epochs for training
nr_epochs = 15

# %
split = 0.10

# The class type (assuming binary classification)
class_type = 0

# latent dimensions for the VAE
latent_dim = 100

# Number of images displayed
n = 1000 

print('Model name is {}, class {}'.format(model_name, class_type))
print('Parameters-->\n Number of epochs: {}\n Batch size: {}\n Validation split: {}\n Latend dim: {}'.format(nr_epochs, batch_size, split, latent_dim))


Model name is CVEA_Project_AI_version_1, class 0
Parameters-->
 Number of epochs: 15
 Batch size: 32
 Validation split: 0.1
 Latend dim: 100


In [22]:
def get_pcam_generators(base_dir, train_percentage=0.2, batch_size_gen=batch_size, class_type=class_type, split=split, img_size=img_size):
    """
    Uses the ImageDataGenerator function from the Keras API to return images in batches,
    train_gen for the training data and val_gen for the validation data.

    Args:
        base_dir (str): Base directory containing the dataset.
        train_percentage (float): Percentage of training data to be used. Default is 0.2.
        batch_size_gen (int): Batch size for data generators. Default is 32.
        class_type (int): Class type to be used. Default is 0.
        split (float): Fraction of training data to be used as validation data. Default is 0.2.
        img_size (int): Size of the images. Default is 96.

    Returns:
        tuple: A tuple containing train_gen and val_gen, both are generators.
    """
    
    # Dataset parameters
    train_path = os.path.join(base_dir, 'train+val', 'train')
    valid_path = os.path.join(base_dir, 'train+val', 'valid')
    RESCALING_FACTOR = 1./255
    
    # Instantiate data generators
    datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

    # Generate data batches for training and validation sets
    train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(img_size, img_size),
                                             batch_size=batch_size_gen,
                                             subset='training',
                                             classes=[str(class_type)],
                                             class_mode='input')

    # Calculate the number of samples for training data based on train_percentage
    num_train_samples = int(train_percentage * len(train_gen))
    
    # Extract desired percentage of training data for training
    train_gen_subset = train_gen

    val_gen = datagen.flow_from_directory(valid_path,
                                           target_size=(img_size, img_size),
                                           batch_size=batch_size_gen,
                                           subset='validation',
                                           class_mode='input',
                                           classes=[str(class_type)],
                                           shuffle=False)
    return train_gen_subset, val_gen


In [14]:
def get_pcam_generators(base_dir, train_percentage=0.2, batch_size_gen=batch_size, class_type=class_type, split=split, img_size=img_size):
    """
    Uses the ImageDataGenerator function from the Keras API to return images in batches,
    train_gen for the training data and val_gen for the validation data.

    Args:
        base_dir (str): Base directory containing the dataset.
        train_percentage (float): Percentage of training data to be used. Default is 0.2.
        batch_size_gen (int): Batch size for data generators. Default is 32.
        class_type (int): Class type to be used. Default is 0.
        split (float): Fraction of training data to be used as validation data. Default is 0.2.
        img_size (int): Size of the images. Default is 96.

    Returns:
        tuple: A tuple containing train_gen and val_gen, both are generators.
    """
    
    # Dataset parameters
    train_path = os.path.join(base_dir, 'train+val', 'train')
    valid_path = os.path.join(base_dir, 'train+val', 'valid')
    RESCALING_FACTOR = 1./255
    
    # Instantiate data generators
    datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

    # Generate data batches for training and validation sets
    train_gen = datagen.flow_from_directory(train_path,
                                                   target_size=(img_size, img_size),
                                                   batch_size=batch_size_gen,
                                                   subset='training',
                                                   classes=[str(class_type)],
                                                   class_mode='input')

    # # Calculate the number of samples for training data based on train_percentage
    # num_train_samples = int(train_percentage * len(train_gen))
    
    # # Extract desired percentage of training data for training
    # train_gen_subset = next(train_gen)[:num_train_samples]

    val_gen = datagen.flow_from_directory(valid_path,
                                                target_size=(img_size, img_size),
                                                batch_size=batch_size_gen,
                                                subset='validation',
                                                class_mode='input',
                                                classes=[str(class_type)],
                                                shuffle=False)
    return train_gen, val_gen


In [4]:
train_gen, val_gen = get_pcam_generators(base_dir, batch_size_gen=batch_size, class_type=i)


print(val_gen)


NameError: name 'i' is not defined

In [23]:
def construct_vae(train_gen, val_gen, weights_filepath, model_name, nr_epochs=1, latent_dim=latent_dim, batch_size=batch_size, img_size=img_size):
    
    # Define input shape and latent dimension
    input_shape = (img_size, img_size, 3)
    
    # Encoder network
    inputs = Input(shape=input_shape)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    shape_before_flattening = K.int_shape(x)
    x = Flatten()(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    

    # Sampling function
    @tf.function
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
        return z_mean + K.exp(z_log_var / 2) * epsilon
    

    # Reparameterization trick
    z = Lambda(sampling)([z_mean, z_log_var])
    

    # Decoder network
    decoder_input = Input(K.int_shape(z)[1:])
    x = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
    x = Reshape(shape_before_flattening[1:])(x)
    x = Conv2DTranspose(128, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2DTranspose(64, (2, 2), activation='relu', padding='same', strides=(2, 2))(x)
    x = Conv2DTranspose(32, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2DTranspose(16, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Define the VAE model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    decoder = Model(decoder_input, x, name='decoder')
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name=model_name)

    # Define the VAE loss function
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= input_shape[0] * input_shape[1] * input_shape[2]
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=1)
    B = 1000   
    vae_loss = K.mean(B * reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.add_metric(kl_loss, name="kl_loss")
    vae.add_metric(reconstruction_loss, name="reconstruction_loss")
    vae.compile(optimizer='adam')

    # Serialize model to JSON
    model_json = vae.to_json() # serialize model to JSON
    with open(model_name, 'w') as json_file:
        json_file.write(model_json)
    
    # Define EarlyStopping callback to stop training when the model stops improving
    early_stopping = EarlyStopping(monitor="val_loss", patience = 5,
                                  restore_best_weights=True)
    
    # Define other callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(log_dir=os.path.join('logs', model_name))
    callbacks_list = [early_stopping, checkpoint, tensorboard]

    # Calculate number of steps per epoch for training and validation sets
    train_steps = train_gen.n // train_gen.batch_size
    val_steps = val_gen.n // val_gen.batch_size
    
    # Fit the VAE model
    vae.fit(train_gen, 
            steps_per_epoch=train_steps, 
            epochs=nr_epochs, 
            batch_size=batch_size, 
            validation_data=val_gen, 
            validation_steps=val_steps,
            callbacks=callbacks_list)
    
    # Returns the trained VAE model
    return vae

In [24]:
def reconstruct_img(vae, val_gen, img_size=img_size):
    
    # Reconstruct images using the trained VAE model
    decoded_imgs = vae.predict(val_gen)
    
    
    # Display the original and reconstructed images
    plt.figure(figsize=(20, 4))
    for i in range(n):
        
        # Display the original image
        ax = plt.subplot(2, n, i + 1)
        img, label = val_gen.next()
        plt.imshow(img[0])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display the reconstructed image
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(img_size, img_size,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()



def generate_new_img(vae, base_dir, class_type=0, num_samples=10, delete_files=True, img_size=img_size):
    
    # Generate new images using random latent vectors
    random_latent_vectors  = np.random.random((num_samples, img_size, img_size, 3))
    decoded_imgs = vae.predict(random_latent_vectors)
    
    # Path to the directory where you want to save the images
    save_dir = base_dir + "/train_new_images/{}/".format(str(class_type))

    # Create the save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Optionally delete existing files in the directory
    for file in os.listdir(save_dir):
        if file.endswith('.jpg') and delete_files == True:
            os.remove(save_dir + file)

    # Iterate through generated images and save them
    for i in range(len(decoded_imgs)):
        img = decoded_imgs[i].reshape(img_size, img_size,3)
        
        # Generate a random name for the image
        random_name = ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ', k=10))
        
        # Save the image with the random name and jpg extension
        plt.imsave(os.path.join(save_dir, random_name + ".jpg"), img)

    print("Images saved to:", save_dir)

In [25]:
# Perform autoencoder for specified classes (for both classes = [0,1])
classes = [0,1]
for i in classes:
    # Get data generators for the specified class
    train_gen, val_gen = get_pcam_generators(base_dir, batch_size_gen=batch_size, class_type=i)
    
    # Construct VAE model and train it
    vae = construct_vae(train_gen, val_gen, weights_filepath, model_name, nr_epochs, latent_dim)
    
    # Reconstruct and display images
    reconstruct_img(vae, val_gen)
    
    # Generate new images and save them
    generate_new_img(vae, base_dir, class_type=i, num_samples=1000)

Found 72000 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Epoch 1/15
   2/2250 [..............................] - ETA: 3:17:05 - loss: 2783876.0000 - kl_loss: 3278.4658 - reconstruction_loss: 2780.5977

KeyboardInterrupt: 

In [ ]:
# Code for the CNN 
#the pathway for the train+val data
path = 'C:\\Users\\20213314\\Documents\\BMT\\Jaar 3\\Project AI\\Data'
# save the model and weights
model_name = 'cnn_model_group_final'



In [ ]:
def get_pcam_generators(base_dir, batch_size_gen=batch_size, class_type=class_type, split=split, img_size=img_sizebase_dir, img_size=img_size,  train_batch_size=5, val_batch_size=32):
    """
    Uses the ImageDataGenerator function from the Keras API to return images in batches,
    train_gen for the training data and val_gen for the validation data.

    Args:
        base_dir (str): Base directory containing the dataset.
        train_batch_size (int): Batch size for training data. Default is 32.
        val_batch_size (int): Batch size for validation data. Default is 32.

    Returns:
        tuple: A tuple containing train_gen and val_gen, both are generators.
    """
    # Dataset parameters
    train_path = os.path.join(base_dir, 'train_new_images')#, 'train')
    valid_path = os.path.join(base_dir, 'train+val', 'valid')
    RESCALING_FACTOR = 1./255
    
    # Instantiate data generators
    datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

    # Generate data batches for training and validation sets
    train_gen = datagen.flow_from_directory(train_path,
                                            target_size=(imgIMAGE_SIZE, IMAGE_SIZE),
                                            batch_size=train_batch_size,
                                            class_mode='binary')

    val_gen = datagen.flow_from_directory(VALID_PATH,
                                          target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                          batch_size=val_batch_size,
                                          class_mode='binary',
                                          shuffle=False)
    
    return train_gen, val_gen

In [ ]:
def get_model(kernel_size=(3,3), pool_size=(4,4), first_filters=32, second_filters=64):
    """
    Generates a convolutional neural network model with configurable parameters.

    Args:
        kernel_size (tuple): Tuple specifying the height and width of the 2D convolution window. Default is (3, 3).
        pool_size (tuple): Tuple specifying the factor by which to downscale in the pooling operation. Default is (4, 4).
        first_filters (int): Number of filters in the first convolutional layer. Default is 32.
        second_filters (int): Number of filters in the second convolutional layer. Default is 64.

    Returns:
        model: A Keras Sequential model.
    """
    # Build the model
    model = Sequential()

    # Add convolutional layers
    model.add(Conv2D(first_filters, kernel_size, activation='relu', padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    model.add(MaxPool2D(pool_size=pool_size))

    model.add(Conv2D(second_filters, kernel_size, activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=pool_size))

    # Flatten the output to feed into dense layers
    model.add(Flatten())

    # Add dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(SGD(learning_rate=0.01, momentum=0.95), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(),'accuracy'])

    return model

def train_model(model, train_gen, val_gen, weights_filepath, model_name, epochs=1):
    """
    Trains the provided model using the given data generators and saves the best weights.

    Args:
        model: The Keras Sequential model to train.
        train_gen: Generator for training data.
        val_gen: Generator for validation data.
        weights_filepath (str): Filepath to save the best weights.
        model_name (str): Name of the model, used for TensorBoard logging.
        epochs (int): Number of epochs to train the model for. Default is 1.

    Returns:
        history: History object containing training metrics.
    """

    model_json = model.to_json() # serialize model to JSON
    with open(model_name, 'w') as json_file:
        json_file.write(model_json)

    # Define callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(log_dir=os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]

    # Train the model
    train_steps = train_gen.n // train_gen.batch_size
    val_steps = val_gen.n // val_gen.batch_size

    history = model.fit(train_gen, steps_per_epoch=train_steps, 
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=epochs,
                        callbacks=callbacks_list)
    return history

def get_fcn_model(first_filters=32, second_filters=64, kernel_size=(3,3), pool_size=(4,4)):
    """
    Generates a fully convolutional neural network model with configurable parameters.

    Args:
        first_filters (int): Number of filters in the first convolutional layer. Default is 32.
        second_filters (int): Number of filters in the second convolutional layer. Default is 64.
        kernel_size (tuple): Tuple specifying the height and width of the 2D convolution window. Default is (3, 3).
        pool_size (tuple): Tuple specifying the factor by which to downscale in the pooling operation. Default is (4, 4).

    Returns:
        model: A Keras Sequential model.
    """
    # Build the model
    model = Sequential()

    # Add convolutional layers
    model.add(Conv2D(first_filters, kernel_size, activation='relu', padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    model.add(MaxPool2D(pool_size=pool_size))

    model.add(Conv2D(second_filters, kernel_size, activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=pool_size))

    model.add(Conv2D(second_filters, kernel_size, activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=pool_size))

    # Output layer
    model.add(Conv2D(1, kernel_size, activation='sigmoid', padding='same'))
    model.add(GlobalAveragePooling2D())

    # Compile the model
    model.compile(SGD(learning_rate=0.01, momentum=0.95), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(),'accuracy'])

    return model

In [ ]:
def calculate_roc_and_auc(model, val_gen):
    """
    Performs the calculations necessary for the ROC curve and its AUC value.

    Args:
        model: The trained Keras model.
        val_gen: Generator for validation data.

    Returns:
        tuple: False positive rate, true positive rate, and AUC value.
    """
    # Predict probabilities for the validation set
    predictions = model.predict(val_gen)

    # Calculate the false positive rate (FPR) and true positive rate (TPR)
    fpr, tpr, thresholds = roc_curve(val_gen.labels, predictions)
    
    # Calculate the area under the ROC curve (AUC)
    roc_auc = auc(fpr, tpr)

    # Print classification report
    predictions[predictions <= 0.5] = 0.
    predictions[predictions > 0.5] = 1.
    print('Classification report:\n', classification_report(val_gen.labels, predictions))
    print('Confusion matrix:\n', confusion_matrix(val_gen.labels, predictions))
    
    return fpr, tpr, roc_auc

def plot_roc_curve(fpr, tpr, roc_auc, type='dense'):
    """
    Plot the ROC curve using the false positive rate, true positive rate, and AUC value.

    Args:
        fpr (array): False positive rate.
        tpr (array): True positive rate.
        roc_auc (float): Area under the ROC curve.
        type (str): Type of model to use ('dense' or 'conv'). Default is 'dense'. 
    """
    # Plot the ROC curve
    if type == 'dense':
        name_type ='connected'
    elif type == 'conv':
        name_type='convolutional'
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='gray', lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve of model with fully {} layers".format(name_type))
    plt.legend(loc="lower right")
    plt.show()

def calculate_and_plot_ROC_AUC(type='dense', model_name='my_model'):
    """
    Combine different functions to create and train the model, and to calculate and plot the ROC curve and AUC value.

    Args:
        type (str): Type of model to use ('dense' or 'conv'). Default is 'dense'.
        model_name (str): Name of the model.

    """
    # Get the model
    if type == 'dense':
        model = get_model()
    elif type == 'conv':
        model = get_fcn_model()

    print('Summary of model:')
    for layer in model.layers:
        print(layer.output_shape)

    # Get the data generators
    train_gen, val_gen = get_pcam_generators(path)
    
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    # Train the model
    train_model(model, train_gen, val_gen, weights_filepath, model_filepath, epochs=3)

    # Load the trained model weights
    model.load_weights(weights_filepath)

    # Evaluate the model
    score = model.evaluate(val_gen)
    print("Loss:", score[0])
    print("Accuracy:", score[1])
    
    # Calculate ROC and AUC
    fpr, tpr, roc_auc = calculate_roc_and_auc(model, val_gen)
    print("AUC:", roc_auc)
    
    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, type)